<a href="https://colab.research.google.com/github/Isaquintero2004/Anal-tica-de-Negocios-2/blob/main/RETO_N%C2%B01_ISABELA_QUINTERO_FERN%C3%81NDEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RETO N°1: MODELO NAIVE BAYES (CLASIFICACIÓN)**
De acuerdo con la base de datos Car Crash, seleccionar los datos para el condado de San Francisco. Predecir si un accidente automovilístico se provocó en una autopista o no (Highway). Para la predicción de los accidentes, es importante tener en cuenta las siguientes variables: Dia de la Semana (WeekDay), Nivel de Impacto (ViolCat), Cantidad de Luz Diurna, Mes, Crash Type (Convertir a Valor Numérico según el Orden Alfabético) y Cielo Despejado. El orden de las variables en las columnas se debe mantener como esta en el archivo de Excel.

De acuerdo con la base de datos anterior, llevar a cabo en el análisis la descripción de cada una de las variables (tipo de datos, categorías), eliminando de plano los registros que poseen datos faltantes.   

0. Se carga las librerias de trabajo

In [ ]:
import numpy as np #Funciones matemáticas
import pandas as pd #Comunicación con excel

1. Se carga la base de datos

In [ ]:
nxl="/content/1. BD2_CarCrash_Int.xlsx"
XDB=pd.read_excel(nxl, sheet_name=0)

#Filtramos los datos para solo obtener información de San Francisco
san_francisco_data=XDB[XDB["County"]=="SAN FRANCISCO"]
san_francisco_data.dropna() #Eliminar datos faltantes, celdas vacías

#Convertir "CrashType" a valores númericos
san_francisco_data.loc[:,"CrashType"]=pd.Categorical(san_francisco_data["CrashType"]).codes

#Se cargan las variables de interés
san_francisco_data=san_francisco_data[["Weekday","ViolCat","ClearWeather","Month","CrashType","Highway","Daylight"]]

#Separamos variables por entrada-salida
XD=san_francisco_data[["Weekday", "ViolCat", "ClearWeather", "Month", "CrashType", "Daylight"]] #Aquí variable de entrada
yd=san_francisco_data[["Highway"]].values.ravel() #Aquí variable de referencia

#Imprimimos la base de datos
print(san_francisco_data)


        Weekday  ViolCat  ClearWeather  Month CrashType  Highway  Daylight
907           7        1             1      3         0        1         1
1580          2        9             1      2         0        0         0
1581          7        3             1      1         0        0         0
1586          7        3             1      3         0        0         0
1587          3        3             1      3         0        0         0
...         ...      ...           ...    ...       ...      ...       ...
112598        5       11             1      8         6        0         0
112599        5       11             1      3         6        0         1
112600        6       10             0      4         6        0         0
112601        4       10             1      4         6        0         0
112602        5       12             1      2         6        0         1

[2081 rows x 7 columns]


2. Implementamos el modelo de Naive Bayes

In [ ]:
#Llamar al modelo Naive Bayes
from sklearn.naive_bayes import GaussianNB

mnb=GaussianNB()
mnb.fit(XD,yd)

#Se obtiene la información
ct=mnb.class_count_
print("Como están separados los datos: \n", ct)
print("El total de los datos es: \n", len(XD))
print("Los datos porcentualmente están divididos en: \n", ct/len(XD))

#Se obtiene la información de los accidentes por categoría
#Media
u=mnb.theta_
print("La media de los accidentes por categoría es: \n", u)

#Desviación estándar
std=mnb.var_  #Varianza es el valor elevado a la 2, para std hay que sacarle esa raíz
std=np.sqrt(std)  #Desviación estándar
print("La desviación estándar de los accidentes por categoría es: \n", std)

#Valores superiores e inferiores de las categorias
print("El limite superior de los valores: \n", u+std)
print("El limite inferior de los valores: \n", u-std)

#Para evaluar un solo accidente
XS=[[5,1,1,3,4,0]]
XS = pd.DataFrame(XS, columns=["Weekday", "ViolCat", "ClearWeather", "Month", "CrashType", "Daylight"])
mnb.predict(XS)
print("La predicción para el accidente según los datos es: \n", mnb.predict(XS))

Como están separados los datos: 
 [1580.  501.]
El total de los datos es: 
 2081
Los datos porcentualmente están divididos en: 
 [0.75925036 0.24074964]
La media de los accidentes por categoría es: 
 [[3.88417722 7.45696203 0.86265823 4.35189873 3.21772152 0.66265823]
 [3.9261477  4.1497006  0.79840319 6.69461078 2.27744511 0.62075848]]
La desviación estándar de los accidentes por categoría es: 
 [[1.95062542 3.52136997 0.34420782 2.3433645  1.87371962 0.47280262]
 [2.01603723 2.16198902 0.40119266 3.44163641 1.05765448 0.48519831]]
El limite superior de los valores: 
 [[ 5.83480263 10.978332    1.20686605  6.69526323  5.09144114  1.13546084]
 [ 5.94218494  6.31168962  1.19959585 10.13624719  3.33509959  1.10595679]]
El limite inferior de los valores: 
 [[1.9335518  3.93559205 0.51845041 2.00853424 1.3440019  0.18985561]
 [1.91011047 1.98771158 0.39721054 3.25297437 1.21979063 0.13556017]]
La predicción para el accidente según los datos es: 
 [0]


3. **Matriz de confusión:** Se procede con la evaluación de desempeño del modelo. Para esto se utilizará la matriz de confusión, para mirar como un modelo de estos funciona o no funciona.

In [ ]:
ydp=mnb.predict(XD) #Se obtiene la predicción de positivos y negativos

#Matriz de confusión
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd, ydp)
print("La matriz de confusión es: \n", cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1] #Enumeramos donde se encuentra cada variable

#Tasa de exactitud
Ex=(VN+VP)/(VP+VN+FP+FN) #En cuanto acierta
print("La exactitud es: \n", Ex)

#Tasa de error
TE=(FP+FN)/(VP+VN+FP+FN) #En cuanto se equivoca
print("La tasa de error es: \n", TE)

#Tasa sensibilidad
Sen=VP/(VP+FN) #Comportamiento de preaprobación créditos
print("La sensibilidad es: \n", Sen)

#Tasa especificidad
Esp=VN/(VN+FP) #Comportamiento de preanegación créditos
print("La especificidad es: \n", Esp)
#El modelo es más bueno considerando los prenegados ya que es 84% en comparación con 63% de aprobados

#Tasa de precisión
Pre=VP/(VP+FP) #Comportamiento de identificación de preaprobación créditos
print("La precisión es: \n", Pre)

#Tasa de precisión negativa
pren=VN/(VN+FN) #Comportamiento de identificación de preanegación créditos
print("La precisión negativa es: \n", pren)

La matriz de confusión es: 
 [[1365  215]
 [ 224  277]]
La exactitud es: 
 0.7890437289764536
La tasa de error es: 
 0.21095627102354636
La sensibilidad es: 
 0.5528942115768463
La especificidad es: 
 0.8639240506329114
La precisión es: 
 0.5630081300813008
La precisión negativa es: 
 0.8590308370044053


# **Análisis de resultados**
De acuerdo con los resultados arrojados por el modelo, se pudo corrobar en la plataforma colab el comportamiento del modelo Naive Bayes de acuerdo con el archivo de excel de referencia acerca de los accidentes automovilísticos ocasionados en diferentes condados.

Para comprender el proceso realizado, primero es importante tener una idea sobre la base de datos con la que se trabajó, la cual contiene el registro de un total de 112660 datos evaluados a partir de ocho variables, de las cuales para la realización del modelo de Naive Bayes se tomaron aquellas correspondientes al condado de San Francisco, al realizar el filtro del condado, el total de datos de la base de datos se redujo a 2081 como lo indica el modelo, los cuales fueron evaluados con seis de las ocho variables totales, las cuales son: **"Weekday"** corresponde al día de la semana en el que el accidente se desenvolvió, asignándole un puntaje de 1 a 7 respectivamente según el día, **"ViolCat"** que nos indica el nivel de impacto de dicho accidente en un rango de 1 a 12 según la severidad del mismo, **"ClearWeather"** es una variable que indica que tan despejado se encontraba el clima o el cielo calificándolo en 0 si no estaba despejado, y 1 si el cielo sí se encontraba despejado, **"Month"** en un rango de 1 a 12 que hace referencia a cada mes del año, **"CrashType"** para indicar el tipo de accidente que se presentó. En la base de datos original este es medido con letras de la A a la G, pero para motivos del modelo esta categorización fue cambiada a número comenzando desde el 0 para representar la A hasta el 6 para representar la G. Y por último, **"Daylight"** para indicar la cantidad de luz diurna al momento del accidente, calificándolo con valores entre 0 y 1 según el caso. Así mismo, como base de datos de referencia y para la predicción del modelo se usó la variable "Highway" la cual clasifica los datos en 0 si el accidente NO fue en una autopista (highway) o 1 si el accidente SÍ fue en una autopista (highway).

Después de comprender el tipo de base de datos empleada podemos evaluar el modelo y los resultados:

Aquí se puede observar que el modelo alcanzó una exactitud cercana al 80%, destacándose el comportamiento del modelo frente a la capacidad que tiene de predecir los accidentes NO ocasionados en la autopista (highway) con una especificidad del 86% en promedio, lo que se ve reforzado con una precisión negativa del 85,9%. Sin embargo, con una precisión cercana al 56% en promedio, el modelo muestra una baja capacidad predictiva frente a los accidentes que pueden ser generados en la autopista (highway), respaldada por la tasa de sensibilidad del 55,2%.

En la base de datos se puede observar que los accidentes automovilísticos (Datos históricos) NO ocasionados en la autopista representan un total del 75,9% (1580 accidentes) de los accidentes totales mientras que los accidentes ocasionados en la autopista tiene un peso de 24% (501 accidentes) del total de los accidentes. Para realizar un análisis de la media obtenida en el modelo, tomaremos dos de las variables y explicaremos como entender dichos resultados para que se demuestre el entendimiento de los mismos:

* **Weekday (Día de la semana):** Las medias son 3.88 para "No Highway" y 3.926 para "Highway". Son muy similares, lo que sugiere que el día de la semana no es un factor determinante para que un accidente ocurra en autopista o no. Esto implica que los accidentes se distribuyen de forma relativamente uniforme a lo largo de la semana en ambos tipos de vías.

* **ViolCat (Nivel de impacto):** La media es 7.457 para "No Highway" y 4.15 para "Highway". Esta es una diferencia significativa. Indica que los accidentes que NO ocurren en autopista tienden a tener un nivel de impacto mayor.

Con lo anterior, podemos tener una idea más clara de como el modelo de Naive Bayes es aplicado a una base de datos que indica accidentes automovilísticos para así tomar decisiones basadas en los resultados obtenidos, que puedan ser precisos y acertados según los datos presentados.

